## Deriving the A_lambda/E(B-V) coefficients needed for dereddening<br>
This notebook shows a quick demo of how to calculate the A_lambda/E(B-V) coefficients needed to remove Galactic dust from the DC2 catalogs.  <br>

In DC2, the CCM model [*reference?*] was assumed when calculating the dust model, we will use the lsst.sims.photUtils package to compute the effective wavelengths for the filters, and then set up the CCM model and calculate Alambda/E(B-V) for each of the LSST filters.<br>

In [ ]:
import os
import GCRCatalogs
import pandas as pd
import numpy as np
import scipy.interpolate
from lsst.sims import photUtils
from lsst.sims.photUtils import BandpassSet

This function grabs the "LSST_THROUGHPUTS_BASELINE" filters sa currently specified in the photUtils package, then computes both lambda_eff and A_lambda/E(B-V) for each of the six LSST filters:

In [ ]:
def compute_alambda_over_ebv(filterset=['u','g','r','i','z','y'],basepath = None):
    """
    compute the effective wavelengths and alambda/E(B-V) values for a set of filters
    We will grab the flat SED from the SIMS library to calculate the CCM dust model 
    that was assumed for DC2, and then grab the baseline ugrizy filters and calculate 
    their effective wavelengths, and evaluate the CCM alam_over_ebv value at those
    wavelengths
    inputs: filterset:
      vector of filters (limited to ugrizy present for the baseline LSST filterset)
    returns:
    lam_eff_list: 
      np 1d array of filter effective wavelengths for the filters
    alam_over_ebv_list:
      np 1d array of alam_over_ebv values for the filters
    """
    lam_eff_list = []
    alam_over_ebv_list = []
    sed_file = os.path.join(os.environ['SIMS_SED_LIBRARY_DIR'],'flatSED','sed_flat.txt.gz')
    sed = photUtils.Sed()
    sed.readSED_flambda(sed_file)
    ax,bx = sed.setupCCM_ab()
    ccm_model = 3.1*ax+bx
    wl = sed.wavelen
    ccm_spline = scipy.interpolate.interp1d(wl,ccm_model,bounds_error=True)
    alam_over_ebv = 3.1*ax+bx
    filterset = ['u','g','r','i','z','y']
    for filter in filterset:
        if basepath is None:
            bp_file = os.path.join(os.environ['LSST_THROUGHPUTS_BASELINE'],'',f'total_{filter}.dat')
        else:
            bp_file = os.path.join(basepath,f'total_{filter}.dat')
        bandpass = photUtils.Bandpass()
        bandpass.readThroughput(bp_file)
        _,leff = bandpass.calcEffWavelen()
        lam_eff_list.append(leff)
        #print(f"band {filter} leff1: {leff:.3f}")
        alam = ccm_spline(leff)
        #print(f"leff for band {filter}: {alam:.3f}")
        alam_over_ebv_list.append(alam)
    return np.array(lam_eff_list),np.array(alam_over_ebv_list)

Let's do a quick check that we are getting the results that we expect.  For DC2 we should get the following for the effective wavelengths and A_lam/E(B-V) values:<br>
u 367.07 nm A_lambda/EBV = 4.812<br>
g 482.69 nm A_lambda/EBV = 3.643<br>
r 622.32 nm A_lambda/EBV = 2.699<br>
i 754.60 nm A_lambda/EBV = 2.063<br>
z 869.01 nm A_lambda/EBV = 1.578<br>
y 971.03 nm A_lambda/EBV = 1.313<br>
If these values do not match those in the next cell, check that the Baseline filter definitions have not changed!

In [ ]:
filterlist = ['u','g','r','i','z','y']
leff_list,alamebv_list = compute_alambda_over_ebv(filterlist)
for filt,leff, alamebv in zip(filterlist,leff_list,alamebv_list):
    print(f"filter {filt} lam_eff: {leff:.2f}nm   alam/E(B-V): {alamebv:.3f}")


These numbers do *not* match, there are very slight differences, as the filters were updated in mid 2019.  The filter curves used for DC2 are released as v1.4 cosmoDC2:
https://github.com/lsst/throughputs/releases/tag/1.4.  For convenience, we have saved a copy of these filters in the data/dc2_throughputs subdirectory.  So, we can calculate the lambda_eff and A_lambda/E(B-V) for these filters:

In [ ]:
filterlist = ['u','g','r','i','z','y']
leff_list,alamebv_list = compute_alambda_over_ebv(filterlist,basepath="data/dc2_throughputs")
for filt,leff, alamebv in zip(filterlist,leff_list,alamebv_list):
    print(f"filter {filt} lam_eff: {leff:.2f}nm   alam/E(B-V): {alamebv:.3f}")

Yes, these numbers do agree.  So, if need A/E(B-V) coefficients for a different filter set, you can use this procedure to derive them.